<a href="https://colab.research.google.com/github/NagaKartheekReddy/NLP-Tasks/blob/main/Q%26ASearchApplication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Question and Answers cognitive search application using haystack library


1.   Dataset : https://www.kaggle.com/stanfordu/stanford-question-answering-dataset
2. Elasticsearch DB to store the processed documents



In [ ]:
!pip install git+https://github.com/deepset-ai/haystack.git
!wget --no-check-certificate https://dl.xpdfreader.com/xpdf-tools-linux-4.03.tar.gz
!tar -xvf xpdf-tools-linux-4.03.tar.gz && sudo cp xpdf-tools-linux-4.03/bin64/pdftotext /usr/local/bin

In [60]:
# import required libraries
from haystack.file_converter.pdf import PDFToTextConverter
from haystack.preprocessor.utils import convert_files_to_dicts
from haystack.preprocessor.preprocessor import PreProcessor

from haystack.document_store.elasticsearch import ElasticsearchDocumentStore

from haystack.retriever.sparse import ElasticsearchRetriever
from haystack.reader.farm import FARMReader
from haystack.finder import Finder

from haystack.pipeline import ExtractiveQAPipeline

In [61]:
from farm.utils import initialize_device_settings

device, n_gpu = initialize_device_settings(use_cuda=True)


02/22/2021 21:47:53 - INFO - farm.utils -   Using device: CUDA 
02/22/2021 21:47:53 - INFO - farm.utils -   Number of GPUs: 1
02/22/2021 21:47:53 - INFO - farm.utils -   Distributed Training: False
02/22/2021 21:47:53 - INFO - farm.utils -   Automatic Mixed Precision: None


In [62]:
# Extracting text data from pdf document
converter = PDFToTextConverter(remove_numeric_tables=True, valid_languages=["en"])
doc_pdf = converter.convert(file_path=f'/content/drive/MyDrive/Colab Notebooks/data_aa.pdf', meta=None)

In [63]:
# Preprocessing the text
preprocessor = PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=True,
    split_by="passage",
    split_length=5,
    split_respect_sentence_boundary=False,
    split_overlap=0
)

processed_docs = preprocessor.process(doc_pdf)

In [64]:
# starting elastic search
! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
! tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
! chown -R daemon:daemon elasticsearch-7.9.2

import os
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['elasticsearch-7.9.2/bin/elasticsearch'],
                   stdout=PIPE, stderr=STDOUT,
                   preexec_fn=lambda: os.setuid(1)  # as daemon
                  )
! sleep 30

In [65]:
document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", index="document")

02/22/2021 21:48:56 - INFO - elasticsearch -   HEAD http://localhost:9200/ [status:200 request:0.030s]
02/22/2021 21:48:56 - INFO - elasticsearch -   HEAD http://localhost:9200/document [status:200 request:0.008s]
02/22/2021 21:48:56 - INFO - elasticsearch -   GET http://localhost:9200/document [status:200 request:0.002s]
02/22/2021 21:48:56 - INFO - elasticsearch -   PUT http://localhost:9200/document/_mapping [status:200 request:0.019s]
02/22/2021 21:48:56 - INFO - elasticsearch -   HEAD http://localhost:9200/label [status:200 request:0.002s]


In [66]:
# Storing processed data docs to elasticsearch DB
document_store.write_documents(processed_docs)

02/22/2021 21:48:59 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:0.788s]


In [67]:
# retriever filter
retriever = ElasticsearchRetriever(document_store=document_store)

In [68]:
# Reader object
reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)

02/22/2021 21:49:04 - INFO - farm.utils -   Using device: CUDA 
02/22/2021 21:49:04 - INFO - farm.utils -   Number of GPUs: 1
02/22/2021 21:49:04 - INFO - farm.utils -   Distributed Training: False
02/22/2021 21:49:04 - INFO - farm.utils -   Automatic Mixed Precision: None
Some weights of RobertaModel were not initialized from the model checkpoint at deepset/roberta-base-squad2 and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
02/22/2021 21:49:15 - WARNING - farm.utils -   ML Logging is turned off. No parameters, metrics or artifacts will be logged to MLFlow.
02/22/2021 21:49:15 - INFO - farm.utils -   Using device: CUDA 
02/22/2021 21:49:15 - INFO - farm.utils -   Number of GPUs: 1
02/22/2021 21:49:15 - INFO - farm.utils -   Distributed Training: False
02/22/2021 21:49:15 - INFO - farm.utils -   Automatic Mixed Precision: None
02/22/2021 

In [69]:
# creating pipeline for search
pipe = ExtractiveQAPipeline(reader, retriever)

In [ ]:
# takes user Question and return Answer
user_query = input('Enter your question:')
prediction = pipe.run(query=user_query, top_k_retriever=1)
print(prediction)

Enter your question:"How old was Peyton Manning when he played in Super Bowl 50?"


02/22/2021 20:59:07 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.008s]


{'query': '"How old was Peyton Manning when he played in Super Bowl 50?"', 'no_ans_gap': 7.22430944442749, 'answers': [{'answer': '39', 'score': 10.544868469238281, 'probability': 0.7888668478477564, 'context': 'ls. He is also the oldest quarterback ever to play in a Super Bowl at age 39. The past record was held by John Elway, who led the Broncos to victory i', 'offset_start': 74, 'offset_end': 76, 'offset_start_in_doc': 1989, 'offset_end_in_doc': 1991, 'document_id': '7d10edb0-86b1-426c-a74b-aa0d850a7130', 'meta': {'_split_id': 0}}, {'answer': '11-year', 'score': -5.48020601272583, 'probability': 0.3351405315653097, 'context': 'o playo games."\n"Carolina su ered a major setback when Thomas Davis, an 11-year veteran who had already overcome three ACL tears in his career, went d', 'offset_start': 72, 'offset_end': 79, 'offset_start_in_doc': 1569, 'offset_end_in_doc': 1576, 'document_id': '7d10edb0-86b1-426c-a74b-aa0d850a7130', 'meta': {'_split_id': 0}}, {'answer': 'Broncos', 'score': -

In [70]:
doc_index = 'docs'
label_index = 'labels'

In [71]:
# evaluation docs loading into elasticsearch
document_store.delete_all_documents(index=doc_index)
document_store.delete_all_documents(index=label_index)
document_store.add_eval_data(filename=f'/content/drive/MyDrive/Colab Notebooks/eval_doc.json', doc_index=doc_index, label_index=label_index)

02/22/2021 21:49:35 - INFO - elasticsearch -   POST http://localhost:9200/docs/_delete_by_query [status:200 request:0.045s]
02/22/2021 21:49:37 - INFO - elasticsearch -   POST http://localhost:9200/labels/_delete_by_query [status:200 request:0.075s]
02/22/2021 21:49:39 - INFO - elasticsearch -   HEAD http://localhost:9200/docs [status:200 request:0.002s]
02/22/2021 21:49:39 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:0.130s]
02/22/2021 21:49:39 - INFO - elasticsearch -   HEAD http://localhost:9200/labels [status:200 request:0.006s]
02/22/2021 21:49:40 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.085s]


In [72]:
retriever = ElasticsearchRetriever(document_store=document_store)

In [73]:
reader = FARMReader("deepset/roberta-base-squad2", top_k_per_candidate=2)

02/22/2021 21:49:46 - INFO - farm.utils -   Using device: CUDA 
02/22/2021 21:49:46 - INFO - farm.utils -   Number of GPUs: 1
02/22/2021 21:49:46 - INFO - farm.utils -   Distributed Training: False
02/22/2021 21:49:46 - INFO - farm.utils -   Automatic Mixed Precision: None
Some weights of RobertaModel were not initialized from the model checkpoint at deepset/roberta-base-squad2 and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
02/22/2021 21:49:57 - WARNING - farm.utils -   ML Logging is turned off. No parameters, metrics or artifacts will be logged to MLFlow.
02/22/2021 21:49:57 - INFO - farm.utils -   Using device: CUDA 
02/22/2021 21:49:57 - INFO - farm.utils -   Number of GPUs: 1
02/22/2021 21:49:57 - INFO - farm.utils -   Distributed Training: False
02/22/2021 21:49:57 - INFO - farm.utils -   Automatic Mixed Precision: None
02/22/2021 

In [ ]:
finder = Finder(reader, retriever)

In [77]:
# finder evaluation results
finder_eval_results = finder.eval(top_k_retriever=1, top_k_reader=10, label_index=label_index, doc_index=doc_index)
finder.print_eval_results(finder_eval_results)


02/22/2021 21:50:16 - INFO - elasticsearch -   POST http://localhost:9200/labels/_search?scroll=1d&size=10000 [status:200 request:0.015s]
02/22/2021 21:50:16 - INFO - elasticsearch -   POST http://localhost:9200/_search/scroll [status:200 request:0.004s]
02/22/2021 21:50:16 - INFO - elasticsearch -   DELETE http://localhost:9200/_search/scroll [status:200 request:0.002s]
02/22/2021 21:50:16 - INFO - elasticsearch -   POST http://localhost:9200/docs/_search [status:200 request:0.014s]
02/22/2021 21:50:16 - INFO - elasticsearch -   POST http://localhost:9200/docs/_search [status:200 request:0.007s]
02/22/2021 21:50:16 - INFO - elasticsearch -   POST http://localhost:9200/docs/_search [status:200 request:0.004s]
02/22/2021 21:50:16 - INFO - elasticsearch -   POST http://localhost:9200/docs/_search [status:200 request:0.004s]
02/22/2021 21:50:16 - INFO - elasticsearch -   POST http://localhost:9200/docs/_search [status:200 request:0.004s]
02/22/2021 21:50:16 - INFO - elasticsearch -   POST


___Retriever Metrics in Finder___
Retriever Recall            : 1.000
Retriever Mean Avg Precision: 1.000
Retriever Mean Reciprocal Rank: 1.000

___Reader Metrics in Finder___
Top-k accuracy
Reader Top-1 accuracy             : 0.941
Reader Top-1 accuracy (has answer): 0.941
Reader Top-k accuracy             : 0.941
Reader Top-k accuracy (has answer): 0.941
Exact Match
Reader Top-1 EM                   : 0.941
Reader Top-1 EM (has answer)      : 0.941
Reader Top-k EM                   : 0.941
Reader Top-k EM (has answer)      : 0.941
F1 score
Reader Top-1 F1                   : 0.941
Reader Top-1 F1 (has answer)      : 0.941
Reader Top-k F1                   : 0.941
Reader Top-k F1 (has answer)      : 0.941

___Time Measurements___
Total retrieve time           : 0.110
Avg retrieve time per question: 0.006
Total reader timer            : 2.476
Avg read time per question    : 0.146
Total Finder time             : 2.615
